# Showcase of Bybit Future - Future calc
The BybitClient has custom methods that I coded, but can also make normal request to the Bybit API with the session attribute.

In [3]:
from Bybit.bybit import BybitClient

# Create a Bybit client
client = BybitClient()

# Get all futures BTC gaps
client.all_gaps()

,Buy,Sell,Gap,Coeff,APR,DaysLeft,CumVolume
0,BTC-18OCT24,BTC-25OCT24,$ 99.00,0.16 %,5.39 %,5,374.97K
1,BTC-18OCT24,BTC-01NOV24,$ 150.50,0.24 %,8.17 %,5,262.28K
2,BTC-18OCT24,BTC-29NOV24,$ 539.50,0.85 %,29.32 %,5,245.15K
3,BTC-18OCT24,BTC-27DEC24,$ 918.00,1.45 %,49.85 %,5,529.22K
4,BTC-18OCT24,BTC-28MAR25,$ 2298.00,3.64 %,124.83 %,5,844.50K
5,BTC-18OCT24,BTC-27JUN25,$ 3657.00,5.79 %,198.64 %,5,699.47K
6,BTC-18OCT24,BTC-26SEP25,$ 5309.50,8.40 %,288.38 %,5,408.28K
7,BTC-25OCT24,BTC-01NOV24,$ 51.50,0.08 %,1.20 %,12,239.94K
8,BTC-25OCT24,BTC-29NOV24,$ 440.50,0.70 %,10.31 %,12,222.81K
9,BTC-25OCT24,BTC-27DEC24,$ 819.00,1.29 %,19.17 %,12,506.88K
